In [4]:
import os

get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from src.chromaDB_setup import *
from src.github_parser import *
from src.embedding import *
import chromadb
from chromadb.config import Settings
import numpy as np

/Users/vl4duu/PycharmProjects/RAG-Repository-Analyzer/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
repo = g.get_repo("vl4duu/vlad-mutilica.space")
# ("khoj-ai/khoj")

In [7]:
repo_files = get_repo_files("vl4duu/vlad-mutilica.space")

In [8]:
chunked_docs = chunk_repository_files(repo_files)

textual_embeddings = np.array([embed_textual_metadata(doc["content"]) for doc in chunked_docs['textual_chunks']])

# Process code chunks with CodeBERT
code_embeddings = np.array([generate_code_embedding(doc["content"]) for doc in chunked_docs['code_chunks']])

embedded_chunks = {
    'textual_embeddings': np.array(textual_embeddings),
    'code_embeddings': np.array(code_embeddings)
}

print(chunked_docs.keys())

dict_keys(['textual_chunks', 'code_chunks'])


In [9]:
chrombaDb_collections = setup_chroma_collections(chunked_docs, embedded_chunks)

ValueError: Expected embeddings to be a list of floats or ints, a list of lists, a numpy array, or a list of numpy arrays, got [[[-0.322937548160553, 0.1757688969373703, 0.5652825832366943, -0.450846403837204, -0.5966501235961914, -0.6812129020690918, -0.2670528292655945, 0.19422674179077148, 0.2698618173599243, 0.4770990312099457, -0.06860408186912537, 0.6261957883834839, -0.2035769373178482, -0.38631632924079895, 0.949104905128479, 0.3404698371887207, 0.4725770950317383, 0.22900891304016113, 0.01042294129729271, 0.410494863986969, -0.3144736886024475, -0.6796550750732422, 0.6257382035255432, -0.5807931423187256, 0.1401326060295105, 0.4597083628177643, -0.08420304954051971, 0.9131944179534912, -0.7839180827140808, 0.7616409659385681, -0.22435764968395233, 0.18457593023777008, 1.5097949504852295, -0.03122314065694809, 0.46716636419296265, -0.47488871216773987, -0.24821092188358307, 0.292488694190979, 0.26235848665237427, -0.7005587220191956, -0.2759028673171997, 1.6153137683868408, -1.

In [ ]:
# Add to ChromaDB
for idx, doc in enumerate(chunked_docs):
    metadata = {"file_name": doc["file_name"]}
    if idx < len(textual_chunks):  # Assuming textual chunks come first
        metadata["content_type"] = "text"
    else:
        metadata["content_type"] = "code"
    collection.add(
        documents=[doc["content"]],
        metadatas=[metadata],
        ids=[f"{doc['file_name']}_{idx}"],
        embeddings=[embeddings[idx].tolist()]
    )


In [ ]:
def retrieve_relevant_chunks(query, top_k=3):
    # Generate query embedding
    query_embedding = embed_textual_metadata(query).tolist()

    # Query ChromaDB
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )

    # Return relevant documents and metadata
    return results["documents"], results["metadatas"]

In [ ]:
query = "What is this repo supposed to do?"
retrieve_relevant_chunks(query)